In [1]:
from utils import *

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from xgboost import XGBRegressor, DMatrix, cv
from xgboost import train as train_xgb

## Data description

In [2]:
macro_df = pd.read_csv('data/macro.csv', parse_dates=['timestamp'])
train_df = pd.read_csv('data/train.csv', index_col='id', parse_dates=['timestamp'])
test_df = pd.read_csv('data/test.csv', index_col='id', parse_dates=['timestamp'])

tverskoe_issue_fix(train_df)
tverskoe_issue_fix(test_df)

Fix:  550
Fix:  149


## 1. Data preprocessing
## I part (encoding and correcting mistakes)

### Macro dataset

In [3]:
macro_df['child_on_acc_pre_school'] = macro_df['child_on_acc_pre_school'].str.replace('#!', 'nan')
for column in macro_df.select_dtypes('object').columns:
    macro_df[column] = macro_df[column].str.replace(',', '.')
    macro_df[column] = macro_df[column].astype(float)

if not len(macro_df.select_dtypes('object').columns):
    print('OK')

OK


### Train dataset

In [4]:
train_df = encode(train_df)

### Test dataset

In [5]:
test_df = encode(test_df)

## II part (Filling missing values)

XGBRegressor model handles `np.NaN` values itself

## 2. Encoding `sub_area` feature

In [6]:
coords_train_df = pd.read_csv('data/geo/train_lat_lon.csv')
coords_train_df.drop(['key', 'tolerance_m'], axis=1, inplace=True)
coords_train_df.index = coords_train_df.id
coords_train_df.drop(['id'], axis=1, inplace=True)
coords_train_df = coords_train_df.sort_index()

coords_test_df = pd.read_csv('data/geo/test_lat_lon.csv')
coords_test_df.drop(['key', 'tolerance_m'], axis=1, inplace=True)
coords_test_df.index = coords_test_df.id
coords_test_df.drop(['id'], axis=1, inplace=True)
coords_test_df = coords_test_df.sort_index()

coords_all_df = pd.concat([coords_train_df, coords_test_df])

In [7]:
train_df['is_train'] = 1
test_df['is_train'] = 0

# coords_df = pd.read_csv('data/coords.csv', index_col='id')
all_df = pd.concat([train_df, test_df])

all_df['latitude'] = coords_all_df['lat']
all_df['longitude'] = coords_all_df['lon']

## 3. Removing outliers

In [8]:
all_df = remove_outliers(all_df)

## 4. Feature engineering

In [9]:
all_df = create_new_features(all_df)

## 5. Removing fake prices

In [10]:
train_df = all_df[all_df['is_train'] == 1].drop(['is_train'], axis=1)
test_df = all_df[all_df['is_train'] == 0].drop(['is_train', 'price_doc'], axis=1)

In [11]:
train_df = remove_fake_prices(train_df)

REMOVED: 35


In [12]:
idx_outliers = np.loadtxt('outliers/idx_outliers_full.txt').astype(int)
train_df = train_df.drop(idx_outliers)

## BONUS: Merging `macro.csv` with main

In [13]:
macro_df['timestamp_day'] = macro_df['timestamp'].dt.day
macro_df['timestamp_month'] = macro_df['timestamp'].dt.month
macro_df['timestamp_year'] = macro_df['timestamp'].dt.year
macro_df = macro_df.drop('timestamp', axis=1)

In [14]:
train_macro_df = train_df.merge(macro_df, on=['timestamp_day', 'timestamp_month', 'timestamp_year'], how='left' )
test_macro_df = test_df.merge(macro_df, on=['timestamp_day', 'timestamp_month', 'timestamp_year'], how='left' )

## 6. Normalizing prices

In [15]:
train_2011_q3_index = train_df.loc[train_df['timestamp_year'] == 2011].loc[train_df['timestamp_month'] >= 7].loc[train_df['timestamp_month'] < 10].index
train_2011_q4_index = train_df.loc[train_df['timestamp_year'] == 2011].loc[train_df['timestamp_month'] >= 10].loc[train_df['timestamp_month'] <= 12].index
train_2012_q1_index = train_df.loc[train_df['timestamp_year'] == 2012].loc[train_df['timestamp_month'] >= 1].loc[train_df['timestamp_month'] < 4].index
train_2012_q2_index = train_df.loc[train_df['timestamp_year'] == 2012].loc[train_df['timestamp_month'] >= 4].loc[train_df['timestamp_month'] < 7].index
train_2012_q3_index = train_df.loc[train_df['timestamp_year'] == 2012].loc[train_df['timestamp_month'] >= 7].loc[train_df['timestamp_month'] < 10].index
train_2012_q4_index = train_df.loc[train_df['timestamp_year'] == 2012].loc[train_df['timestamp_month'] >= 10].loc[train_df['timestamp_month'] <= 12].index
train_2013_q1_index = train_df.loc[train_df['timestamp_year'] == 2013].loc[train_df['timestamp_month'] >= 1].loc[train_df['timestamp_month'] < 4].index
train_2013_q2_index = train_df.loc[train_df['timestamp_year'] == 2013].loc[train_df['timestamp_month'] >= 4].loc[train_df['timestamp_month'] < 7].index
train_2013_q3_index = train_df.loc[train_df['timestamp_year'] == 2013].loc[train_df['timestamp_month'] >= 7].loc[train_df['timestamp_month'] < 10].index
train_2013_q4_index = train_df.loc[train_df['timestamp_year'] == 2013].loc[train_df['timestamp_month'] >= 10].loc[train_df['timestamp_month'] <= 12].index
train_2014_q1_index = train_df.loc[train_df['timestamp_year'] == 2014].loc[train_df['timestamp_month'] >= 1].loc[train_df['timestamp_month'] < 4].index
train_2014_q2_index = train_df.loc[train_df['timestamp_year'] == 2014].loc[train_df['timestamp_month'] >= 4].loc[train_df['timestamp_month'] < 7].index
train_2014_q3_index = train_df.loc[train_df['timestamp_year'] == 2014].loc[train_df['timestamp_month'] >= 7].loc[train_df['timestamp_month'] < 10].index
train_2014_q4_index = train_df.loc[train_df['timestamp_year'] == 2014].loc[train_df['timestamp_month'] >= 10].loc[train_df['timestamp_month'] <= 12].index
train_2015_q1_index = train_df.loc[train_df['timestamp_year'] == 2015].loc[train_df['timestamp_month'] >= 1].loc[train_df['timestamp_month'] < 4].index
train_2015_q2_index = train_df.loc[train_df['timestamp_year'] == 2015].loc[train_df['timestamp_month'] >= 4].loc[train_df['timestamp_month'] < 7].index

In [18]:
train_q_idx = [train_2011_q3_index, train_2011_q4_index, train_2012_q1_index,
               train_2012_q2_index, train_2012_q3_index, train_2012_q4_index,
               train_2013_q1_index, train_2013_q2_index, train_2013_q3_index,
               train_2013_q4_index, train_2014_q1_index, train_2014_q2_index,
               train_2014_q3_index, train_2014_q4_index, train_2015_q1_index,
               train_2015_q2_index]

In [19]:
sum(len(q) for q in train_q_idx) == len(train_df)

True

In [22]:
train_q_idx.pop()

Int64Index([28763, 28764, 28765, 28766, 28767, 28768, 28770, 28771, 28772,
            28773,
            ...
            30464, 30465, 30466, 30467, 30468, 30469, 30470, 30471, 30472,
            30473],
           dtype='int64', name='id', length=1612)

In [26]:
sum(len(q) for q in train_q_idx) == len(train_df) - len(train_df.loc[train_2015_q2_index])

True

In [27]:
train_df['average_q_price'] = 1
train_df.loc[train_2015_q2_index, 'average_q_price'] = 1
base_price = (train_df.loc[train_2015_q2_index].price_doc / train_df.loc[train_2015_q2_index].full_sq).mean()
for q in train_q_idx:
    train_df.loc[q, 'average_q_price'] = base_price / (train_df.loc[q].price_doc / train_df.loc[q].full_sq).mean() 


In [29]:
train_df['price_doc'] = train_df['price_doc'] * train_df['average_q_price']

## 7. Modeling

### `product_type == 'Investment'`

In [30]:
train_df_0 = train_df[train_df['product_type'] == 0]
test_df_0 = test_df[test_df['product_type'] == 0]

X = train_df_0.drop(['sub_area', 'price_doc', 'average_q_price'], axis=1).copy()
y = np.log1p(train_df_0['price_doc'])
X_test_0 = test_df_0.drop(['sub_area'], axis=1).copy()

In [31]:
# train_macro_df_0 = train_macro_df[train_macro_df['product_type'] == 0]
# test_macro_df_0 = test_macro_df[test_macro_df['product_type'] == 0]

# X = train_macro_df_0.drop(['sub_area', 'price_doc'], axis=1).copy()
# y = np.log1p(train_macro_df_0['price_doc'])
# X_test_0 = test_macro_df_0.drop(['sub_area'], axis=1).copy()

In [32]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_val.shape

((13416, 306), (3354, 306))

In [33]:
params = {'objective': 'reg:squarederror',
          'tree_method': 'gpu_hist',
          'booster': 'gbtree',
          'n_estimators': 10000,
          'base_score': 7,
          'learning_rate': 0.05,
          'max_depth': 4,
          'min_child_weight': 7,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'reg_lambda': 5,
          'reg_alpha': 1,
          'eval_metric': 'rmse',
          'seed': 42,
          'nthread': -1
          }

model_0 = XGBRegressor(**params)

model_0.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=50)

[0]	validation_0-rmse:8.42466	validation_1-rmse:8.41869
[1]	validation_0-rmse:8.00437	validation_1-rmse:7.99884
[2]	validation_0-rmse:7.60510	validation_1-rmse:7.59989
[3]	validation_0-rmse:7.22581	validation_1-rmse:7.22098
[4]	validation_0-rmse:6.86551	validation_1-rmse:6.86055
[5]	validation_0-rmse:6.52323	validation_1-rmse:6.51853
[6]	validation_0-rmse:6.19809	validation_1-rmse:6.19327
[7]	validation_0-rmse:5.88922	validation_1-rmse:5.88466
[8]	validation_0-rmse:5.59582	validation_1-rmse:5.59122
[9]	validation_0-rmse:5.31713	validation_1-rmse:5.31266
[10]	validation_0-rmse:5.05237	validation_1-rmse:5.04804
[11]	validation_0-rmse:4.80088	validation_1-rmse:4.79659
[12]	validation_0-rmse:4.56196	validation_1-rmse:4.55773
[13]	validation_0-rmse:4.33502	validation_1-rmse:4.33085
[14]	validation_0-rmse:4.11944	validation_1-rmse:4.11553
[15]	validation_0-rmse:3.91464	validation_1-rmse:3.91082
[16]	validation_0-rmse:3.72011	validation_1-rmse:3.71639
[17]	validation_0-rmse:3.53535	validation

XGBRegressor(base_score=7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eval_metric='rmse',
             gamma=0, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=4, min_child_weight=7, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=8,
             nthread=-1, num_parallel_tree=1, random_state=42, reg_alpha=1,
             reg_lambda=5, scale_pos_weight=1, seed=42, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [34]:
y_pred_train = np.abs(model_0.predict(X_train))
y_pred_val = np.abs(model_0.predict(X_val))
print('___________________________________')
print('      RMSLE\n')
print('      TRAIN_0:', np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
print(' VALIDATION_0:', np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))
print('___________________________________')
print('      MAE\n')
print('      TRAIN_0:', mean_absolute_error(np.expm1(y_train), np.expm1(y_pred_train)))
print(' VALIDATION_0:', mean_absolute_error(np.expm1(y_val), np.expm1(y_pred_val)))

___________________________________
      RMSLE

      TRAIN_0: 0.09893957092224799
 VALIDATION_0: 0.14107959976505138
___________________________________
      MAE

      TRAIN_0: 558156.3281134614
 VALIDATION_0: 841785.9267754137


### `product_type == 'OwnerOccupier'`

In [35]:
train_df_1 = train_df[train_df['product_type'] == 1]
test_df_1 = test_df[test_df['product_type'] == 1]

X = train_df_1.drop(['sub_area', 'price_doc', 'average_q_price'], axis=1).copy()
y = np.log1p(train_df_1['price_doc'])
X_test_1 = test_df_1.drop(['sub_area'], axis=1).copy()

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_val.shape

((8793, 306), (2199, 306))

In [37]:
params = {'objective': 'reg:squarederror',
          'n_estimators': 10000,
          'tree_method': 'gpu_hist',
          'booster': 'gbtree',
          'base_score': 7,
          'learning_rate': 0.05,
          'max_depth': 4,
          'min_child_weight': 7,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'reg_lambda': 15,
          'reg_alpha': 1,
          'eval_metric': 'rmse',
          'seed': 42,
          'nthread': -1
          }

model_1 = XGBRegressor(**params)

model_1.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=50)

[0]	validation_0-rmse:8.20090	validation_1-rmse:8.18484
[1]	validation_0-rmse:7.79277	validation_1-rmse:7.77666
[2]	validation_0-rmse:7.40508	validation_1-rmse:7.38892
[3]	validation_0-rmse:7.03681	validation_1-rmse:7.02059
[4]	validation_0-rmse:6.68699	validation_1-rmse:6.67071
[5]	validation_0-rmse:6.35470	validation_1-rmse:6.33836
[6]	validation_0-rmse:6.03906	validation_1-rmse:6.02287
[7]	validation_0-rmse:5.73919	validation_1-rmse:5.72328
[8]	validation_0-rmse:5.45433	validation_1-rmse:5.43855
[9]	validation_0-rmse:5.18368	validation_1-rmse:5.16786
[10]	validation_0-rmse:4.92655	validation_1-rmse:4.91054
[11]	validation_0-rmse:4.68226	validation_1-rmse:4.66647
[12]	validation_0-rmse:4.45018	validation_1-rmse:4.43421
[13]	validation_0-rmse:4.22970	validation_1-rmse:4.21384
[14]	validation_0-rmse:4.02025	validation_1-rmse:4.00421
[15]	validation_0-rmse:3.82126	validation_1-rmse:3.80585
[16]	validation_0-rmse:3.63224	validation_1-rmse:3.61677
[17]	validation_0-rmse:3.45267	validation

XGBRegressor(base_score=7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eval_metric='rmse',
             gamma=0, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=4, min_child_weight=7, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=8,
             nthread=-1, num_parallel_tree=1, random_state=42, reg_alpha=1,
             reg_lambda=15, scale_pos_weight=1, seed=42, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [38]:
y_pred_train = np.abs(model_1.predict(X_train))
y_pred_val = np.abs(model_1.predict(X_val))
print('___________________________________')
print('      RMSLE\n')
print('      TRAIN_1:', np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
print(' VALIDATION_1:', np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))
print('___________________________________')
print('      MAE\n')
print('      TRAIN_1:', mean_absolute_error(np.expm1(y_train), np.expm1(y_pred_train)))
print(' VALIDATION_1:', mean_absolute_error(np.expm1(y_val), np.expm1(y_pred_val)))

___________________________________
      RMSLE

      TRAIN_1: 0.052984128662542235
 VALIDATION_1: 0.08534366567217072
___________________________________
      MAE

      TRAIN_1: 265241.88770262076
 VALIDATION_1: 400707.6173989804


In [39]:
test_df_nan = test_df[test_df['product_type'].isna()]
X_test_nan = test_df_nan.drop(['sub_area'], axis=1).copy()

In [40]:
submission = pd.read_csv('data/submits/sample_submission.csv', index_col='id')
pred_0 = np.expm1(model_0.predict(X_test_0))
pred_1 = np.expm1(model_1.predict(X_test_1))
pred_nan = np.expm1(model_1.predict(X_test_nan))

if len(pred_0[pred_0 < 0]) or len(pred_1[pred_1 < 0]) or len(pred_nan[pred_nan < 0]):
    print('WARNING: NEGATIVE PREDICTIONS')

In [73]:
submission.loc[X_test_0.index, 'price_doc'] = 0.89*pred_0 # 0.87
submission.loc[X_test_1.index, 'price_doc'] = 0.94*pred_1 # 0.93
submission.loc[X_test_nan.index, 'price_doc'] = pred_nan # 0.95
submission.to_csv('data/submits/submission.csv', index='id')

In [74]:
!kaggle competitions submit -c sberbank-russian-housing-market -f "data/submits/submission.csv" -m "XGBRegressor normalized"

100%|████████████████████████████████████████| 121k/121k [00:01<00:00, 74.4kB/s]
Successfully submitted to Sberbank Russian Housing Market

In [45]:
def get_place(my_score):
    df = pd.read_csv('data/submits/publicleaderboard.csv')
    scores = df['Score'].values
    scores = np.append(scores, my_score)
    scores = np.sort(scores)
    print(f'{np.where(scores == my_score)[0][0]} / {len(scores)}')

In [75]:
get_place(0.31625)

1282 / 3266
